In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("volkert.csv", header = None)

X = df.iloc[:, 2:].to_numpy()
Y = df.iloc[:, 1].to_numpy()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim import Optimizer
import timeit

In [ ]:
class FullData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


full_data = FullData(torch.from_numpy(X).float(), torch.from_numpy(Y).long())

In [ ]:
train_size = int(0.8 * len(full_data))
test_size = len(full_data) - train_size
train_data, test_data = torch.utils.data.random_split(full_data, [train_size, test_size])

train_loader = DataLoader(dataset=train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64)

In [ ]:
class MultiClassification(nn.Module):
    def __init__(self):
        super(MultiClassification, self).__init__()

        self.layer_1 = nn.Linear(180, 256) 
        self.layer_2 = nn.Linear(256, 256)
        self.layer_3 = nn.Linear(256, 256)
        self.layer_out = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        x = self.layer_1(inputs)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.relu(x)
        
        x = self.layer_3(x)
        x = self.relu(x)
        
        x = self.layer_out(x)
        return x

In [ ]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model = MultiClassification()
model.to(device)

criterion = nn.CrossEntropyLoss()
torch.save(model.state_dict(), "volkert_mlp.pth")

In [ ]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc

In [ ]:
learning_rate_list = [0.01, 0.02, 0.05, 0.08, 0.1, 0.005, 0.002, 0.001]

In [ ]:
sgd_data = []

for learning_rate in learning_rate_list:

    model = MultiClassification()
    model.load_state_dict(torch.load("volkert_mlp.pth"))

    model.to(device)
    
    optimizer = optim.SGD(model.parameters(), lr = learning_rate)

    timelist_sgd = []
    losslist_sgd_train = []
    acclist_sgd_train = []
    losslist_sgd_test = []
    acclist_sgd_test = []

    start_time = timeit.default_timer()

    for e in range(1, 100+1):
        epoch_loss = 0
        epoch_acc = 0

        model.train()
        for X_batch, y_batch in train_loader:

            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            y_pred = model(X_batch)

            loss = criterion(y_pred, y_batch)
            acc = multi_acc(y_pred, y_batch)

            loss.backward(create_graph = True)

            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        with torch.no_grad():

            val_epoch_loss = 0
            val_epoch_acc = 0

            model.eval()
            for X_val_batch, y_val_batch in test_loader:
                X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)

                y_val_pred = model(X_val_batch)

                val_loss = criterion(y_val_pred, y_val_batch)
                val_acc = multi_acc(y_val_pred, y_val_batch)

                val_epoch_loss += val_loss.item()
                val_epoch_acc += val_acc.item()


        if e % 50 == 0:
            print(f'Epoch {e+0:03}: | train Loss: {epoch_loss/len(train_loader):.5f} | train Acc: {epoch_acc/len(train_loader):.3f}')
            print(f'Epoch {e+0:03}: | test Loss: {val_epoch_loss/len(test_loader):.5f} | test Acc: {val_epoch_acc/len(test_loader):.3f}')


        epoch_time = timeit.default_timer()
        losslist_sgd_train.append(epoch_loss/len(train_loader))
        timelist_sgd.append(epoch_time - start_time)
        acclist_sgd_train.append(epoch_acc/len(train_loader))
        losslist_sgd_test.append(val_epoch_loss/len(test_loader))
        acclist_sgd_test.append(val_epoch_acc/len(test_loader))
        
    sgd_data.append(timelist_sgd)
    sgd_data.append(losslist_sgd_train)
    sgd_data.append(acclist_sgd_train)
    sgd_data.append(losslist_sgd_test)
    sgd_data.append(acclist_sgd_test)

In [ ]:
adam_data = []

for learning_rate in learning_rate_list:

    model = MultiClassification()
    model.load_state_dict(torch.load("volkert_mlp.pth"))

    model.to(device)
    
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)

    timelist_adam = []
    losslist_adam_train = []
    acclist_adam_train = []
    losslist_adam_test = []
    acclist_adam_test = []

    start_time = timeit.default_timer()

    for e in range(1, 200+1):
        epoch_loss = 0
        epoch_acc = 0

        model.train()
        for X_batch, y_batch in train_loader:

            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            y_pred = model(X_batch)

            loss = criterion(y_pred, y_batch)
            acc = multi_acc(y_pred, y_batch)

            loss.backward(create_graph = True)

            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        with torch.no_grad():

            val_epoch_loss = 0
            val_epoch_acc = 0

            model.eval()
            for X_val_batch, y_val_batch in test_loader:
                X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)

                y_val_pred = model(X_val_batch)

                val_loss = criterion(y_val_pred, y_val_batch)
                val_acc = multi_acc(y_val_pred, y_val_batch)

                val_epoch_loss += val_loss.item()
                val_epoch_acc += val_acc.item()


        if e % 50 == 0:
            print(f'Epoch {e+0:03}: | train Loss: {epoch_loss/len(train_loader):.5f} | train Acc: {epoch_acc/len(train_loader):.3f}')
            print(f'Epoch {e+0:03}: | test Loss: {val_epoch_loss/len(test_loader):.5f} | test Acc: {val_epoch_acc/len(test_loader):.3f}')


        epoch_time = timeit.default_timer()
        losslist_adam_train.append(epoch_loss/len(train_loader))
        timelist_adam.append(epoch_time - start_time)
        acclist_adam_train.append(epoch_acc/len(train_loader))
        losslist_adam_test.append(val_epoch_loss/len(test_loader))
        acclist_adam_test.append(val_epoch_acc/len(test_loader))
        
    adam_data.append(timelist_adam)
    adam_data.append(losslist_adam_train)
    adam_data.append(acclist_adam_train)
    adam_data.append(losslist_adam_test)
    adam_data.append(acclist_adam_test)

In [ ]:
def group_product(xs, ys):
    
    return sum([torch.sum(x * y) for (x, y) in zip(xs, ys)])

def normalization(v):
    # normalize a vector
    
    s = group_product(v, v)
    s = s**0.5
    s = s.cpu().item()
    v = [vi / (s + 1e-6) for vi in v]
    return v

class NysHessianpartial():
    
    def __init__(self, rank, rho):
        self.rank = rank
        # rho is the regularization in Nystrom sketch
        self.rho = rho
    
    def get_params_grad(self, model):
        # get parameters and differentiation
        params = []
        grads = []
        for param in model.parameters():
            if not param.requires_grad:
                continue
            params.append(param)
            grads.append(0. if param.grad is None else param.grad + 0.)
        return params, grads
    
    def update_Hessian(self, X_batch, y_batch, model, criterion, device):
        
        shift = 0.001
        # get the model parameters and gradients
        params, gradsH = self.get_params_grad(model)
        # remember the size for each group of parameters
        self.size_vec = [p.size() for p in params]
        # store random gaussian vector to a matrix
        test_matrix = []
        # Hessian vector product
        hv_matrix = []
        
        for i in range(self.rank):
            # generate gaussian random vector
            v = [torch.randn(p.size()).to(device) for p in params]
            # normalize
            v = normalization(v)
            # zero vector to store the shape
            hv_add = [torch.zeros(p.size()).to(device) for p in params]
        
            # update hessian with a subsample batch
            
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            model.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward(create_graph=True)
            params, gradsH = self.get_params_grad(model)
            # calculate the Hessian vector product
            hv = torch.autograd.grad(gradsH, params, grad_outputs=v,only_inputs=True,retain_graph=True)
            # add initial shift
            for i in range(len(hv)):
                hv_add[i].data = hv[i].data.add_(hv_add[i].data)    
                hv_add[i].data = hv_add[i].data.add_(v[i].data * torch.tensor(shift)) 
            
            # reshape the Hessian vector product into a long vector
            hv_ex = torch.cat([gi.view(-1) for gi in hv_add])
            # reshape the random vector into a long vector
            test_ex = torch.cat([gi.view(-1) for gi in v])
            
            # append long vectors into a large matrix
            hv_matrix.append(hv_ex)
            test_matrix.append(test_ex)
        
        # assemble the large matrix
        hv_matrix_ex = torch.column_stack(hv_matrix)
        test_matrix_ex = torch.column_stack(test_matrix)
        # calculate Omega^T * A * Omega for Cholesky
        choleskytarget = torch.mm(test_matrix_ex.t(), hv_matrix_ex)
        # perform Cholesky, if fails, do eigendecomposition
        # the new shift is the abs of smallest eigenvalue (negative) plus the original shift
        try:
            C_ex = torch.linalg.cholesky(choleskytarget)
        except:
            # eigendecomposition, eigenvalues and eigenvector matrix
            eigs, eigvectors = torch.linalg.eigh(choleskytarget)
            shift = shift + 1.1 * torch.abs(torch.min(eigs))
            # add shift to eigenvalues
            eigs = eigs + shift
            # put back the matrix for Cholesky by eigenvector * eigenvalues after shift * eigenvector^T 
            C_ex = torch.linalg.cholesky(torch.mm(eigvectors, torch.mm(torch.diag(eigs), eigvectors.T)))
        
        # triangular solve
        # B_ex = torch.linalg.solve_triangular(C_ex, hv_matrix_ex, upper = False, left = False)
        B_ex = torch.triangular_solve(hv_matrix_ex.t(), C_ex.t(), upper = True)
        # SVD
        # U, S, V = torch.linalg.svd(B_ex, full_matrices = False)
        U, S, V = torch.linalg.svd(B_ex[0].t(), full_matrices = False)

        self.U = U
        self.S = torch.max(torch.square(S) - torch.tensor(shift), torch.tensor(0.0))

class NysHessianOpt(Optimizer):
    r"""Implements NysHessian.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate
        rank (int): sketch rank
        rho: regularization
    """
    def __init__(self, params, rank = 100, rho = 0.1):
        # initialize the optimizer    
        defaults = dict(rank = rank, rho = rho)
        self.nysh = NysHessianpartial(rank, rho)
        super(NysHessianOpt, self).__init__(params, defaults)
         
    def step(self, lr):
        # one step update
        for group in self.param_groups:
            rho = group['rho']
            # compute gradient as a long vector
            g = torch.cat([p.grad.view(-1) for p in group['params']])
            # calculate the search direction by Nystrom sketch and solve
            UTg = torch.mv(self.nysh.U.t(), g) 
            g_new = torch.mv(self.nysh.U, (self.nysh.S + rho).reciprocal() * UTg) + g / rho - torch.mv(self.nysh.U, UTg) / rho            
            ls = 0
            # update model parameters
            for p in group['params']:
                gp = g_new[ls:ls+torch.numel(p)].view(p.shape)
                ls += torch.numel(p)
                p.data.add_(-lr * gp)

In [ ]:
hes_interval = 2 * len(train_loader) - 1
# update Hessian and Nystrom sketch every couple of steps

skechysgd_data = []

for learning_rate in learning_rate_list:

    model = MultiClassification()
    model.load_state_dict(torch.load("volkert_mlp.pth"))

    model.to(device)

    optimizer = NysHessianOpt(model.parameters())

    hes_iter = 0

    timelist_skechysgd = []
    losslist_skechysgd_train = []
    acclist_skechysgd_train = []
    losslist_skechysgd_test = []
    acclist_skechysgd_test = []

    lr = torch.tensor(learning_rate)

    start_time = timeit.default_timer()


    for e in range(1, 200+1):
        epoch_loss = 0
        epoch_acc = 0

        model.train()
        for X_batch, y_batch in train_loader:

            if hes_iter % hes_interval == 0:
                # update Hessian and sketch
                optimizer.nysh.update_Hessian(X_batch, y_batch, model, criterion, device)

            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            y_pred = model(X_batch)

            loss = criterion(y_pred, y_batch)
            acc = multi_acc(y_pred, y_batch)

            loss.backward()

            optimizer.step(lr)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            hes_iter += 1

        with torch.no_grad():

            val_epoch_loss = 0
            val_epoch_acc = 0

            model.eval()
            for X_val_batch, y_val_batch in test_loader:
                X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)

                y_val_pred = model(X_val_batch)

                val_loss = criterion(y_val_pred, y_val_batch)
                val_acc = multi_acc(y_val_pred, y_val_batch)

                val_epoch_loss += val_loss.item()
                val_epoch_acc += val_acc.item()


        if e % 50 == 0:
            print(f'Epoch {e+0:03}: | train Loss: {epoch_loss/len(train_loader):.5f} | train Acc: {epoch_acc/len(train_loader):.3f}')
            print(f'Epoch {e+0:03}: | test Loss: {val_epoch_loss/len(test_loader):.5f} | test Acc: {val_epoch_acc/len(test_loader):.3f}')


        epoch_time = timeit.default_timer()
        losslist_skechysgd_train.append(epoch_loss/len(train_loader))
        timelist_skechysgd.append(epoch_time - start_time)
        acclist_skechysgd_train.append(epoch_acc/len(train_loader))
        losslist_skechysgd_test.append(val_epoch_loss/len(test_loader))
        acclist_skechysgd_test.append(val_epoch_acc/len(test_loader))
    
    skechysgd_data.append(timelist_skechysgd)
    skechysgd_data.append(losslist_skechysgd_train)
    skechysgd_data.append(acclist_skechysgd_train)
    skechysgd_data.append(losslist_skechysgd_test)
    skechysgd_data.append(acclist_skechysgd_test)